In [1]:
#0.98907 on Kaggle Digit Recognizer MNIST data Competition

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
X_train = train.drop(labels = ["label"],axis = 1) 
Y_train = train["label"]

#normalize pixels
X_train = X_train / 255.0
test = test / 255.0

#reshape
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)
Y_train = to_categorical(Y_train, num_classes = 10)

In [5]:
training_data, validation_data, tY, vY = train_test_split(X_train, Y_train, test_size = 0.1, random_state=2)

In [6]:
#Basic Convolutional Neural Network which predicts image value ranging from 0-9

tf.random.set_seed(12)
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

2022-07-18 13:10:56.625121: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 576)               0

In [8]:
model.fit(training_data, tY, epochs = 10, validation_data = (validation_data,vY))

Epoch 1/10
1182/1182 [==============================] - 21s 17ms/step - loss: 0.1845 - accuracy: 0.9423 - val_loss: 0.0720 - val_accuracy: 0.9769
Epoch 2/10
1182/1182 [==============================] - 20s 17ms/step - loss: 0.0554 - accuracy: 0.9821 - val_loss: 0.0583 - val_accuracy: 0.9838
Epoch 3/10
1182/1182 [==============================] - 19s 16ms/step - loss: 0.0387 - accuracy: 0.9875 - val_loss: 0.0580 - val_accuracy: 0.9833
Epoch 4/10
1182/1182 [==============================] - 20s 17ms/step - loss: 0.0289 - accuracy: 0.9904 - val_loss: 0.0405 - val_accuracy: 0.9883
Epoch 5/10
1182/1182 [==============================] - 21s 17ms/step - loss: 0.0221 - accuracy: 0.9924 - val_loss: 0.0408 - val_accuracy: 0.9883
Epoch 6/10
1182/1182 [==============================] - 21s 18ms/step - loss: 0.0196 - accuracy: 0.9935 - val_loss: 0.0490 - val_accuracy: 0.9871
Epoch 7/10
1182/1182 [==============================] - 21s 18ms/step - loss: 0.0143 - accuracy: 0.9956 - val_loss: 0.0385 -

In [9]:
logit=model.predict(training_data)
preds = np.argmax(logit,axis = 1).tolist()
tmpY = np.argmax(tY,axis = 1).tolist()
print("Accuracy = {}".format(accuracy_score(tmpY, preds)))
print()
logit=model.predict(validation_data)
preds=np.argmax(logit,axis = 1).tolist()
vmpY = np.argmax(vY,axis = 1).tolist()
print("Accuracy = {}".format(accuracy_score(vmpY, preds)))
print()
logit=model.predict(test)
y_pred=np.argmax(logit,axis = 1).tolist()

1182/1182 [==============================] - 5s 4ms/step
Accuracy = 0.9980952380952381

132/132 [==============================] - 1s 5ms/step
Accuracy = 0.9866666666666667

875/875 [==============================] - 4s 5ms/step


In [10]:
ans = pd.DataFrame()
ans['ImageId'] = range(1,28001)
ans['Label'] = y_pred
ans

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [11]:
ans.to_csv('preds.csv',index=False)